In [ ]:
# Downloading preprocessed data, embeddings, models, and configs
!wget http://boytsov.info/datasets/knn4qa_demo2019-04-10.tar.bz2 

In [ ]:
# Unpack the data 
!cd knn4qa ; tar jxvf ../knn4qa_demo2019-04-10.tar.bz2 

In [ ]:
# Create Lucene index
!cd knn4qa ; scripts/index/create_lucene_index.sh squad

In [ ]:
# Create forward index
# First zero means bag-of-words only
# Second zero means we create a binary on-disk version of the index (slower to re-rank)
!cd knn4qa ; scripts/index/create_inmemfwd_index.sh squad 0 0

In [ ]:
# Export index to NMSLIB
!mkdir knn4qa/nmslib/squad
!cd knn4qa ; scripts/data/run_export_to_nmslib_sparse.sh \
-extr_json misc/cosine_fusion_dim=100_squad_emb_unlemm_epoch1.json \
-model_file  misc/out_squad_train_20.model  \
-memindex_dir memfwdindex/squad/ \
-embed_dir WordEmbeddings/squad/ \
-out_file nmslib/squad/cosine_fusion_dim=100_squad_emb_unlemm_epoch1.json_sparse.bin


In [ ]:
# Finally starting the query server on port 8080 and creating the index using HNSW
!cd nmslib/query_server/cpp_client_server ; \
    ./query_server -i ../../../knn4qa/nmslib/squad/cosine_fusion_dim=100_squad_emb_unlemm_epoch1.json_sparse.bin \
    -s cosinesimil_sparse_bin_fast -m hnsw -c M=20,efConstruction=200 -t ef=200 -p 8080 